# CS4001/4042 Assignment 1, Part B, Q2
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [1]:
# !pip install pytorch-widedeep
# !pip install --upgrade torch --user

In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score

C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")
<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()


>Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [3]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here
train_data = df[df['year'] <= 2020]
test_data = df[df['year'] >= 2021]

<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()


>Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [9]:
# TODO: Enter your code here
categorical_cols = [('month', df['month'].nunique()), ('town', df['town'].nunique()), ('flat_model_type', df['flat_model_type'].nunique()), ('storey_range', df['storey_range'].nunique())]
continuous_cols = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']

# cat_embed_cols = [(column_name, embed_dim), ...]
tab_preprocessor = TabPreprocessor(cat_embed_cols=categorical_cols, continuous_cols=continuous_cols)

X_tab = tab_preprocessor.fit_transform(train_data)

#TabMlp(column_idx, cat_embed_input=None, cat_embed_dropout=0.1, use_cat_bias=False, cat_embed_activation=None, continuous_cols=None, cont_norm_layer='batchnorm', embed_continuous=False, cont_embed_dim=32, cont_embed_dropout=0.1, use_cont_bias=True, cont_embed_activation=None, mlp_hidden_dims=[200, 100], mlp_activation='relu', mlp_dropout=0.1, mlp_batchnorm=False, mlp_batchnorm_last=False, mlp_linear_first=False)

#https://pytorch-widedeep.readthedocs.io/en/latest/quick_start.html (reference code from here)
tab_mlp = TabMlp(
    column_idx=tab_preprocessor.column_idx, 
    cat_embed_input=tab_preprocessor.cat_embed_input,
    continuous_cols=continuous_cols,
    mlp_hidden_dims=[200, 100]
)

model = WideDeep(deeptabular=tab_mlp)

#train and validate
trainer = Trainer(model, cost_fn="rmse",num_workers=0, metrics=[R2Score])
trainer.fit(
    X_tab=X_tab,
    target=train_data['resale_price'].values,
    n_epochs=100,
    batch_size=64,
)












C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\pytorch_widedeep\preprocessing\tab_preprocessor.py:334: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")
epoch 100: 100%|██████████| 1366/1366 [00:19<00:00, 71.68it/s, loss=5.74e+4, metrics={'r2': 0.8511}]


>Report the test RMSE and the test R2 value that you obtained.

In [12]:
# TODO: Enter your code here
from sklearn.metrics import mean_squared_error, r2_score
X_tab_test = tab_preprocessor.transform(test_data)
preds = trainer.predict(X_tab=X_tab_test, batch_size=64)

print("RMSE:", mean_squared_error(preds, test_data['resale_price'].values, squared=False))
print("R^2:", r2_score(preds, test_data['resale_price'].values))

predict: 100%|██████████| 1128/1128 [00:08<00:00, 128.55it/s]

RMSE: 105928.49407185947
R^2: 0.524054047867564
